In [2]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow.keras import regularizers
import tensorflow as tf
import tensorflow as tf
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [3]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
X_train = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/trainX.npy')
y_train = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/trainy.npy')
X_test = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/testX.npy')
y_test = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


Mounted at /content/drive
Mounted at /content/gdrive
(33104, 128, 3) (33104, 118) (3740, 128, 3) (3740, 118)


In [4]:
# reshape data into time steps of sub-sequences
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
n_steps, n_length = 4, 32
X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, n_features))
X_test = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))

In [5]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [6]:
model2 = Sequential()
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu'), input_shape=(None,n_length,n_features)))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Dropout(0.5)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Flatten()))
model2.add(LSTM(100))
model2.add(Dropout(0.5))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(n_outputs, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, None, 28, 64)     1024      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, None, 24, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, None, 20, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, None, 16, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, None, 16, 64)     0

In [7]:
# fit network
history = model2.fit(X_training, y_training, epochs=10, verbose=False, validation_data=(X_validation, y_validation), batch_size=100)

In [8]:
# evaluate model
loss, accuracy = model2.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9405
Testing Accuracy: 0.8684


In [11]:
y_pred = model2.predict(X_test, verbose=False)
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)

In [19]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred )
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred, average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred, average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred, average='micro')
print('F1 score: %f' % f1)

Accuracy: 0.868449
Precision: 0.868449
Recall: 0.868449
F1 score: 0.868449
